# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment, ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import pandas as pd
import os
import joblib

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'capstone-gridstability-hd'

exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

print(ws)

Workspace name: quick-starts-ws-187289
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-187289
Workspace.create(name='quick-starts-ws-187289', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-187289')


In [3]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
env = Environment.get(workspace = ws, name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [6]:
src = ScriptRunConfig(source_directory = ".",
                      script = "train.py",
                      compute_target = "my-cluster",
                      environment = env
                     )

## Dataset

### Overview

The dataset "Electrical Grid Stability Simulated Data" was obtained from the UCI Machine Learning Repository and can be found [here](https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+#).

The dataset consists of 10000 observations of an electric power grid with four nodes in total, one with electricity generation and three with electricity consumption:

![Diagram](/img/0-Diagram.png)

The dataset contains the following features (node 1 refers to the electricty producer, whereas nodes 2 to 4 refers to the electricity consumer):

    * tau: reaction time of each participant (1 to 4).
    * p: nominal power consumed or generated of each node (1 to 4). A negative value indicates a net consumption, whereas a positive value indicates net generation.
    * g: coefficient (gamma) proportional to price elasticity (1 to 4).
    * stab: the maximal real part of the characteristic equation root. A positive value indicates that the system is linearly unstable.
    * stabf: the stability label of the system. This is a categorical feature: stable/unstable.

The aim of this project is to develop a machine learning model that can predict the stability of the system based on the features of the dataset.
Therefore, it is a classification problem in which we want to predict the stabf value.

In [7]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.config',
 '.ipynb_aml_checkpoints',
 'automl.ipynb',
 'automl.ipynb.amltmp',
 'automl.log',
 'automl_errors.log',
 'azureml_automl.log',
 'best_automl_model.pkl',
 'data',
 'data.json',
 'endpoint.py',
 'endpoint.py.amltmp',
 'environment.yml',
 'gridstability-runs',
 'hyperparameter_tuning.ipynb',
 'hyperparameter_tuning.ipynb.amltmp',
 'outputs',
 'score.py',
 'train.py',
 'training',
 '__pycache__']

In [8]:
ds = pd.read_csv("data/gridstability.csv")

In [9]:
ds.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


## Hyperdrive Configuration

HyperDrive is used to tune the hyperparameters of a Logistic Regression model, which are Regularization Strength (C) and Maximum Number of Iteratations (max_iter). A Logistic Regression is a good choice for this classification problem in order to stablish a baseline.

Random Parameter Sampling randomly selects hyperparameters to evaluate, which makes the hyperparameter tuning convergence fast.

Bandit Policy is an early termination policy based on slack factor/slack amount and evaluation interval. Any run that doesn't fall within the slack factor or slack amount of the evaluation metric with respect to the best performing run will be terminated.

In this problem, we want to maximize the accuracy of the model.

In [10]:
# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
                             "--max_iter":choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=1)

In [11]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             policy=policy,
                             max_total_runs=16,
                             max_concurrent_runs=4)

## Run Details

In [12]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hd_config)

# The widget RunDetails can be used to monitor the run
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_80f36491-f79a-414e-b025-27baae78a62f
Web View: https://ml.azure.com/runs/HD_80f36491-f79a-414e-b025-27baae78a62f?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-187289/workspaces/quick-starts-ws-187289&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-22T16:10:32.047117][API][INFO]Experiment created<END>\n""<START>[2022-02-22T16:10:32.772140][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-22T16:10:33.472760][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_80f36491-f79a-414e-b025-27baae78a62f
Web View: https://ml.azure.com/runs/HD_80f36491-f79a-414e-b025-27baae78a62f?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-187289/workspaces/quick-starts-ws-187289&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_80f36491-f79a-414e-b025-27baae78a62f',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-22T16:10:31.732077Z',
 'endTimeUtc': '2022-02-22T16:18:44.62793Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '20cda461-aaee-4079-9534-2a9dfd2eb3a9',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '44',
  'score': '0.817',
  'best_child_run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187289.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_80f36491-f79a-414e-b025-27baae78a62f/azureml-logs/hyperdrive.txt?sv=2

In [14]:
print(hd_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_13', 'hyperparameters': '{"--C": 1, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}, {'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_0', 'hyperparameters': '{"--C": 20, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}, {'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_6', 'hyperparameters': '{"--C": 200, "--max_iter": 50}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_15', 'hyperparameters': '{"--C": 200, "--max_iter": 200}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_14', 'hyperparameters': '{"--C": 200, "--max_iter": 100}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_11', 'hyperparameters': '{"--C": 10, "--max_iter": 200}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_80

## Best Model

In [15]:
print(hd_run.get_children_sorted_by_primary_metric(top=1, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_80f36491-f79a-414e-b025-27baae78a62f_13', 'hyperparameters': '{"--C": 1, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}]


In [16]:
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_file_names()

['outputs/.amlignore',
 'outputs/best_model_automl.pkl',
 'outputs/model.pkl',
 'outputs/model_automl.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [17]:
best_run.get_metrics()

{'Regularization Strength:': 20.0, 'Max iterations:': 50, 'Accuracy': 0.817}

In [18]:
best_run.get_details()

{'runId': 'HD_80f36491-f79a-414e-b025-27baae78a62f_0',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-22T16:13:03.663756Z',
 'endTimeUtc': '2022-02-22T16:13:07.379446Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '20cda461-aaee-4079-9534-2a9dfd2eb3a9',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '20', '--max_iter', '50'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'my-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'en

In [21]:
best_run.register_model(model_name = 'best_model_hd', model_path='outputs/model.pkl')

Model(workspace=Workspace.create(name='quick-starts-ws-187289', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-187289'), name=best_model_hd, id=best_model_hd:3, version=3, tags={}, properties={})